In [11]:
import sys
import os
import importlib
from collections import Counter
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# --- 1. Path Setup ---
# Absolute path to repo root (adjust if necessary)
repo_root = "/files/pixlball"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root) 

# --- 2. Project Module Imports ---
# Import all project modules using clean names
import src.config as config
import src.dataset as dataset
import src.train as train
import src.evaluate as evaluate
import src.data as data
import src.losses as losses
import src.model as model
import src.utils as utils

# --- 3. Module Reloading (CRITICAL for Notebook Development) ---
# Reload dependencies in order: Config/Utils -> Data/Losses/Model -> Train/Dataset/Evaluate
importlib.reload(config)
importlib.reload(utils)
importlib.reload(data)
importlib.reload(model)
importlib.reload(losses) 
importlib.reload(dataset)
importlib.reload(train)
importlib.reload(evaluate)

# --- 4. Direct Imports (For clean code in subsequent cells) ---
# Import essential classes and functions needed for the pipeline steps

# Configuration
from src.config import DEVICE 

# Data/Dataset Classes
from src.dataset import PitchDatasetMultiTask, TemporalPitchDataset, ContextPitchDatasetMultiTask, FusionPitchDataset

# Training Functions
from src.train import train_model_lstm_threat

# Evaluation/Helpers
from src.evaluate import evaluate_model_lstm_threat
from src.losses import get_model_criteria
from src.model import HybridCNN_LSTM_Threat
from src.utils import get_sequence_lengths

# --- Final Check ---
print(f"Using device: {DEVICE}")

Using device: cpu


In [2]:
data_events = pd.read_parquet(os.path.join(repo_root, "data", "events_data.parquet"), engine="fastparquet")
data_360 = pd.read_parquet(os.path.join(repo_root, "data", "sb360_data.parquet"), engine="fastparquet")

In [3]:
admin_events = [
        'Starting XI', 'Half Start', 'Half End', 'Player On', 'Player Off',
        'Substitution', 'Tactical Shift', 'Referee Ball-Drop', 'Injury Stoppage',
        'Bad Behaviour', 'Shield'
    ]

cleaned_df = data.drop_events(data_events, rows_to_drop=admin_events)

1278 events.


In [4]:
# -----------------------------
# Example usage
# -----------------------------
columns_to_drop = ['clearance_body_part',
                   'clearance_head',
                   'clearance_left_foot',
                   'clearance_other',
                   'clearance_right_foot',
                   'shot_technique',
                   'substitution_replacement_id',
                   'substitution_replacement',
                   'substitution_outcome',
                   'shot_saved_off_target',
                   'pass_miscommunication',
                   'goalkeeper_shot_saved_off_target',
                   'goalkeeper_punched_out',
                   'shot_first_time',
                   'shot_first_time',
                   'shot_body_part',
                   'related_events',
                   'pass_shot_assist', 
                   'pass_straight', 
                   'pass_switch', 
                   'pass_technique', 
                   'pass_through_ball',
                   'goalkeeper_body_part',
                   'goalkeeper_end_location', 
                   'goalkeeper_outcome', 
                   'goalkeeper_position', 
                   'goalkeeper_technique', 
                   'goalkeeper_type', 
                   'goalkeeper_penalty_saved_to_post', 
                   'goalkeeper_shot_saved_to_post', 
                   'goalkeeper_lost_out', 
                   'goalkeeper_Clear', 
                   'goalkeeper_In Play Safe',
                   'shot_key_pass_id',
                   'shot_one_on_one',
                   'shot_end_location',
                   'shot_type',
                   'pass_angle',
                   'pass_body_part',
                   'pass_type',
                   'pass_length',
                   'pass_outswinging',
                   'pass_inswinging',
                   'pass_cross', 
                   'pass_cut_back', 
                   'pass_deflected', 
                   'pass_goal_assist', 
                   'pass_recipient', 
                   'pass_recipient_id', 
                   'pass_assisted_shot_id', 
                   'pass_no_touch', 
                   'pass_end_location', 
                   'pass_aerial_won',
                   'pass_height',
                   'substitution_outcome_id',
                   'tactics',
                   'block_deflection',
                   'dribble_no_touch',
                   'shot_open_goal', 
                   'shot_saved_to_post',
                   'shot_redirect', 
                   'shot_follows_dribble',
                   'period',
                   'injury_stoppage_in_chanin',
                   'block_save_block',
                   'ball recovery_offensive',


                   ]
cleaned_df = data.drop_columns(cleaned_df, columns_to_drop)

# add lookahead outcome
df_with_targets = data.assign_lookahead_outcomes(cleaned_df, lookahead=6)


counts of each outcome nn_target
Keep Possession    71251
Lose Possession    28252
Shot                4830
Name: count, dtype: int64


# Prepare 360 Data

In [5]:
df_360 = data.assign_grid_cells(data_360)
nn_final = data.aggregate_nn_layers_vectorized(df_360)

# Finalize NN Df

In [6]:
nn_dataset = data.prepare_nn_dataset(df_with_targets, nn_final, target_cols=['nn_target', 'goal_flag'], context_cols = True, keep_context_ids = True ) # adjust cols depending on model

# Neural Network final Data Prep

In [7]:
context_cols = [
    'under_pressure', 
    'counterpress', 
    'dribble_nutmeg'
]

# Impute NaN values with 0.0 (float)
# This assumes NaN means the event was NOT under pressure, NOT a counterpress, etc.
nn_dataset[context_cols] = nn_dataset[context_cols].fillna(0.0)


target_map = {"Keep Possession": 0, "Lose Possession": 1, "Shot": 2}

# Apply mapping
nn_dataset['nn_target_int'] = nn_dataset['nn_target'].map(target_map)

# Check
print(nn_dataset[['nn_target', 'nn_target_int']].head())

         nn_target  nn_target_int
0  Keep Possession              0
1  Keep Possession              0
2  Keep Possession              0
3  Keep Possession              0
4  Keep Possession              0


# The LSTM

In [8]:
print(f"Using device: {DEVICE}")

#layer_columns = ["ball_layer", "teammates_layer", "opponents_layer"]

# Get your existing targets
event_targets = nn_dataset['nn_target_int'].values
goal_flags = nn_dataset['goal_flag'].values.astype(np.float32) # Ensure targets are float

# Weights
event_counts = Counter(event_targets)
total_events = len(event_targets)

# Using inverse frequency: total / count
class_weights_event = torch.tensor(
    [total_events / event_counts.get(c, 1) for c in range(len(event_counts))],
    dtype=torch.float32
).to(DEVICE)

STABLE_GOAL_POS_WEIGHT = 5.0
goal_pos_weight = torch.tensor(STABLE_GOAL_POS_WEIGHT, dtype=torch.float32).to(config.DEVICE)

print(f"Goal Positive Weight (0/1 ratio): {goal_pos_weight.item():.2f}")


Using device: cpu
Goal Positive Weight (0/1 ratio): 5.00


In [9]:
# -----------------------------
# Prepare Sequential Dataset
# -----------------------------
print("Preparing 5D Sequential Dataset for LSTM...")

nn_dataset = data.prepare_nn_dataset(df_with_targets, nn_final, target_cols=['nn_target', 'goal_flag', 'possession']) # adjust cols depending on model

target_map = {"Keep Possession": 0, "Lose Possession": 1, "Shot": 2}

# Apply mapping
nn_dataset['nn_target_int'] = nn_dataset['nn_target'].map(target_map)

# Check
print(nn_dataset[['nn_target', 'nn_target_int']].head())

windows = data.build_temporal_windows_with_mask(nn_dataset)

# Assuming 'windows' variable holds the output of data.build_temporal_windows_with_mask()
# Shape: (Num_Events, T, 4, H, W)
temporal_dataset = TemporalPitchDataset(
    windows=windows, 
    event_labels=event_targets, 
    goal_flags=goal_flags
)



Preparing 5D Sequential Dataset for LSTM...
         nn_target  nn_target_int
0  Keep Possession              0
1  Keep Possession              0
2  Keep Possession              0
3  Keep Possession              0
4  Keep Possession              0


In [12]:
# -----------------------------
# Train Hybrid CNN-LSTM Model
# -----------------------------
print("Starting training for Hybrid CNN-LSTM...")
lstm_model = train_model_lstm_threat(
    dataset=temporal_dataset, 
    event_class_weights=class_weights_event, 
    goal_pos_weight=goal_pos_weight
)
print("Hybrid CNN-LSTM Training complete.")



Hybrid LSTM Threat Epoch 1:   0%|          | 13/9069 [00:00<01:09, 129.46it/s, event_loss=1.17, loss=2.1]   

Starting training for Hybrid CNN-LSTM...


Hybrid LSTM Threat Epoch 2: 100%|██████████| 9069/9069 [00:59<00:00, 151.28it/s, event_loss=1.22, loss=2.12]  

Hybrid CNN-LSTM Training complete.


In [13]:
# -----------------------------
# Evaluate LSTM Model
# -----------------------------
print("\nEvaluating LSTM Model on training data...")
metrics = evaluate_model_lstm_threat(lstm_model, temporal_dataset)
# print(lstm_metrics)


Evaluating LSTM Model on training data...

--- Event Outcome Metrics ---
Event Accuracy: 0.6909141029882016
Event Balanced Accuracy: 0.3333333333333333
Event Confusion Matrix:
 [[62659     0     0]
 [23584     0     0]
 [ 4447     0     0]]
              precision    recall  f1-score   support

           0       0.69      1.00      0.82     62659
           1       0.00      0.00      0.00     23584
           2       0.00      0.00      0.00      4447

    accuracy                           0.69     90690
   macro avg       0.23      0.33      0.27     90690
weighted avg       0.48      0.69      0.56     90690


--- Goal Prediction (xG) Metrics ---
Goal Accuracy: 0.8796941758488869
Goal Balanced Accuracy: 0.5
Goal AUC-ROC Score: 0.5146293217132045
Goal Confusion Matrix:
 [[3912    0]
 [ 535    0]]
              precision    recall  f1-score   support

         0.0       0.88      1.00      0.94      3912
         1.0       0.00      0.00      0.00       535

    accuracy           

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

print("--- Event Classification Probabilities (P_outcome) ---")
event_probs = metrics['event_probs']
print(f"Shape of Event Probabilities (N, 3): {event_probs.shape}")

# Average predicted probability for each class (overall confidence)
avg_P_keep = np.mean(event_probs[:, 0])
avg_P_lose = np.mean(event_probs[:, 1])
avg_P_shot = np.mean(event_probs[:, 2])

print(f"Average Predicted P(Keep Possession): {avg_P_keep:.4f}")
print(f"Average Predicted P(Lose Possession): {avg_P_lose:.4f}")
print(f"Average Predicted P(Shot): {avg_P_shot:.4f}")

print("\n--- Goal Prediction Probabilities (xG) ---")
goal_probs = metrics['goal_probs']
print(f"Number of Shots Evaluated: {goal_probs.shape[0]}")

# Average Predicted xG
avg_xg = np.mean(goal_probs)
print(f"Average Predicted xG per Shot: {avg_xg:.4f}")

# Total Predicted xG (sum of all probabilities for the shot events)
total_xg = np.sum(goal_probs)
print(f"Total Predicted xG for all Shots: {total_xg:.2f}")

# The actual number of goals scored in the test set (True Goals)
true_goals = np.sum(metrics['goal_labels'])
print(f"Actual Goals Scored (True Goals): {true_goals:.2f}")

# Print AUC Score (Should be in your metrics dictionary now)
print(f"Goal Prediction AUC-ROC Score: {metrics.get('goal_auc', 'N/A')}")